# Dekompozycja QR

In [66]:
using LinearAlgebra # Biblioteka potrzebna żeby zwracać R jako macierz górnotrójkątną

# ------------------------------------------------------
#               Algorytm Gramma Schmidta
# ------------------------------------------------------

function grammSchmidt(mat)
    # Funkcje pomocnicze
    function vectorNorm(vec1, vec2)
        return transpose(vec1)*(vec2)
    end
    
    function vecProjection(u, vec)
        return vectorNorm(u, vec)/vectorNorm(u,u) * u
    end
    
    function vecNormalize(vec)
        return sqrt(1 / vectorNorm(vec, vec)) * vec
    end

    baseU = Matrix{Float64}(undef, size(mat, 1), 0)
    baseE = Matrix{Float64}(undef, size(mat, 1), 0)

    # wyliczanie bazy E oraz U
    for i in 1:size(mat, 1)
        u = mat[:, i]
        for j in 1:(i-1)
            u = u - vecProjection(baseU[:, j], mat[:, i])
        end

        baseU = hcat(baseU, u)
        baseE = hcat(baseE, vecNormalize(u))
    end
    
    # wyliczanie macierzy R
    R = Matrix{Float64}(undef, size(mat, 1), size(mat, 1))
    for i in 1:size(mat, 1)
        for j in 1:size(mat, 1)
            R[i, j] = vectorNorm(baseE[:, i], mat[:, j])
        end
    end

    baseE, UpperTriangular(R)
end

# Prosta funkcja porównująca macierze
# Zwraca parę liczb: maksymalny i średni błąd macierzy result względem macierzy dest
function CompareMatrix(dest, result)
    maxErr, sumErr = 0, 0
    matSize = size(result, 1)

    for i in 1:matSize
        for j in 1:matSize
            nErr = abs(dest[i, j] - result[i, j])
            sumErr = sumErr + nErr
            maxErr = maxErr > nErr ? maxErr : nErr
        end
    end

    maxErr, sumErr/(matSize*matSize)
end

# Funckja oceniająca jakość algorytmu rozkładu QRalgorithm macierzy mat
# Używa Compare Matrix by określić jak dobrze Q^T Q = Id i QR = mat
function AlgorithmQuality(mat, QRalgoritm; extended = false)
    println("===============================\nQR comparison")
    if extended; println("A = ",mat); end

    q, r =  QRalgoritm(mat)

    id = transpose(q)*q
    em, ea = CompareMatrix(I, id)

    if extended; println("Q = ",q); end
    if extended; println("Q^T*Q = ", id); end
    println("Average Error: ",ea,"\nMax Error: ",em)

    qr = q*r
    em, ea = CompareMatrix(mat, qr)
    if extended; println("R = ",r); end
    if extended; println("QR = ",qr); end
    println("Average Error: ",ea,"\nMax Error: ",em)
    print("\n")
end

AlgorithmQuality([1 0; 6 5], grammSchmidt);
AlgorithmQuality([1 0 3; 6 5 7; 5 7 8], grammSchmidt);


QR comparison
Average Error: 4.163336342344337e-17
Max Error: 8.326672684688674e-17
Average Error: 3.3306690738754696e-16
Max Error: 8.881784197001252e-16

QR comparison
Average Error: 1.4186183092432555e-16
Max Error: 4.163336342344337e-16
Average Error: 1.0608797790862607e-15
Max Error: 4.884981308350689e-15



In [100]:

function JordanBlockGenerator(size, lambda)
    R = Matrix{Float64}(undef, size, 0)
    for i in 1:size 
        R = hcat(R, rand(size) * lambda .- (lambda/2))
        #R = hcat(R, [(j == i) ? Float64(1.0) : (abs(i - j) > 0) ? (i - j)*lambda : Float64(0.0) for j in 1:size]) 
    end
    return R
end 
R = (JordanBlockGenerator(30, 10^10))

#for i in 1:10
#    print(R[:, i], "\n")
#end

AlgorithmQuality(R, grammSchmidt);

QR comparison
Average Error: 4.897224159005642e-16
Max Error: 1.2831877291517952e-14
Average Error: 1.0955804545018407e-6
Max Error: 2.2411346435546875e-5

